本章主要参考TensorFlow官方网站上的新手入门和扩展教程，讲解TensorFlow的基本概念。本章从系统结构、设计理念、编程模型、常用API、存储和加载模型、线程及队列、加载数据、自定义操作等多方面进行讲解，相信通过本章的学习，读者会对TensorFlow的全貌有一个基本的认识。本章的学习对理解TensorFlow的原理和实战非常重要，读者需要用心揣摩。

## 4.1 系统架构

图4-1给出的是TensorFlow的系统架构，自底向上分为设备层和网络层、数据操作层、图计算层、API层、应用层，其中设备层和网络层、数据操作层、图计算层是TensorFlow的核心层。

![4-1](4-attach/4-1.png)

下面就自底向上详细介绍一下TensorFlow的系统架构。最下层是网络通信层和设备管理层。网络通信层gRPC（google Remote Procedure Call Protocol）和远程直接数据存取（Remote Direct Memory Access，RDMA），这都是在分布式计算时需要用到的。设备管理层包括TensorFlow分别在CPU、GPU、FPGA等设备上的实现，也就是对上层提供了一个统一的接口，使上层只需要处理卷积等逻辑，而不需要关心在硬件上的卷积的实现过程。

其上是数据操作层，主要包括卷积函数、激活函数等操作（参见4.7节）。再往上是图计算层，也是我们要了解的核心，包含本地计算图和分布式计算图的实现（本章会做基础知识的梳理，包括图的创建、编译、优化和执行，本书的“实战篇”会介绍图的具体应用，第15章会介绍分布式计算图的实现）。再往上是API层和应用层（4.4节和4.7节会介绍重点常用的API的Python实现以及一些其他语言的实现，本书的“实战篇”会重点讲解调用API层对深度学习各种网络模型的实现）。

## 4.2 设计理念

TensorFlow的设计理念主要体现在以下两个方面。

（1）将图的定义和图的运行完全分开。因此，TensorFlow被认为是一个“符号主义”的库。

我们知道，编程模式通常分为命令式编程（imperative style programming）和符号式编程（symbolic style programming）。命令式编程就是编写我们理解的通常意义上的程序，很容易理解和调试，按照原有逻辑执行。符号式编程涉及很多的嵌入和优化，不容易理解和调试，但运行速度相对有所提升。现有的深度学习框架中，Torch是典型的命令式的，Caffe、MXNet采用了两种编程模式混合的方法，而TensorFlow完全采用符号式编程。

符号式计算一般是先定义各种变量，然后建立一个数据流图，在数据流图中规定各个变量之间的计算关系，最后需要对数据流图进行编译，但此时的数据流图还是一个空壳儿，里面没有任何实际数据，只有把需要运算的输入放进去后，才能在整个模型中形成数据流，从而形成输出值。

例如：

In [1]:
t = 8 + 9
print(t)

17


在传统的程序操作中，定义了t的运算，在运行时就执行了，并输出17。而在TensorFlow中，数据流图中的节点，实际上对应的是TensorFlow API中的一个操作，并没有真正去运行：

In [2]:
import tensorflow as tf
t = tf.add(8, 9)
print(t) # 输出Tensor("Add:0", shape=(), dtype=int32)

/usr/lib64/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Couldn't import dot_parser, loading of dot files will not be possible.


Tensor("Add:0", shape=(), dtype=int32)


定义了一个操作，但实际上并没有运行。

（2）TensorFlow中涉及的运算都要放在图中，而图的运行只发生在会话（session）中。开启会话后，就可以用数据去填充节点，进行运算；关闭会话后，就不能进行计算了。因此，会话提供了操作运行和Tensor求值的环境。例如：

In [3]:
import tensorflow as tf

# 创建图
a = tf.constant([1.0, 2.0])
b = tf.constant([3.0, 4.0])
c = a * b

# 创建会话
sess = tf.Session()

# 计算c
print(sess.run(c)) # 进行矩阵乘法，输出[3., 8.]
sess.close()

[3. 8.]


了解了设计理念，接下来看一下TensorFlow的编程模型。

## 4.3 编程模型

TensorFlow是用数据流图做计算的，因此我们先创建一个数据流图（也称为网络结构图），如图4-2所示，看一下数据流图中的各个要素。

![4-2](4-attach/4-2.png)

图4-2讲述了TensorFlow的运行原理。图中包含输入（input）、塑形（reshape）、Relu层（Relu layer）、Logit层（Logit layer）、Softmax、交叉熵（cross entropy）、梯度（gradient）、SGD训练（SGD Trainer）等部分，是一个简单的回归模型。

它的计算过程是，首先从输入开始，经过塑形后，一层一层进行前向传播运算。Relu层（隐藏层）里会有两个参数，即$W_{h1}$和$b_{h1}$，在输出前使用ReLU（Rectified Linear Units）激活函数做非线性处理。然后进入Logit层（输出层），学习两个参数$W_{sm}$和$b_{sm}$。用Softmax来计算输出结果中各个类别的概率分布。用交叉熵来度量两个概率分布（源样本的概率分布和输出结果的概率分布）之间的相似性。然后开始计算梯度，这里是需要参数$W_{h1}$、$b_{h1}$、$W_{sm}$和$b_{sm}$，以及交叉熵后的结果。随后进入SGD训练，也就是反向传播的过程，从上往下计算每一层的参数，依次进行更新。也就是说，计算和更新的顺序为$b_{sm}$、$W_{sm}$、$b_{h1}$和$W_{h1}$。

故名思义，TensorFlow是指“张量的流动”。TensorFlow的数据流图是由节点（node）和边（edge）组成的有向无环图（directed acycline graph，DAG）。TensorFlow由Tensor和Flow两部分组成，Tensor（张量）代表了数据流图中的边，而Flow（流动）这个动作就代表了数据流图中节点所做的操作。

### 4.3.1 边

TensorFlow的边有两种连接关系：数据依赖和控制依赖。其中，实线边表示数据依赖，代表数据，即张量。任意维度的数据统称为张量。在机器学习算法中，张量在数据流图中从前往后流动一遍就完成了一次前向传播（forward propagation），而残差从后向前流动一遍就完成了一次反向传播（backward propagation）。

还有一种特殊边，一般画为虚线边，称为控制依赖（control dependency），可以用于控制操作的运行，这被用来确保happens-before关系，这类边上没有数据流过，但源节点必须在目的节点开始执行前完成执行。常用代码如下：

``` python
tf.Graph.control_dependencies(control_inputs)
```

TensorFlow支持的张量具有表4-1所示的数据属性。

![表4-1](4-attach/t4-1.png)

有关图及张量的实现的源代码均位于tensorflow-1.1.0/tensorflow/python/framework/ops.py，后面会详细讲。

### 4.3.2 节点

图中的节点又称为算子，它代表一个操作（operation，OP），一般用来表示施加的数学运算，也可以表示数据输入（feed in）的起点以及输出（push out）的终点，或者是读取/写入持久变量（persistent variable）的终点。表4-2列举了一些TensorFlow实现的算子。算子支持表4-1所示的张量的各种数据属性，并且需要在建立图的时候确定下来。

![表4-2](4-attach/t4-2.png)

与操作相关的代码位于tensorflow-1.1.0/tensorflow/python/ops/目录下。以数学运算为例，代码为上述目录下的math_ops.py，里面定义了add、subtract、multiply、scalar_mul、div、divide、truediv、floordiv等数学运算，每个函数里面调用了gen_math_ops.py中的方法，这个文件是在编译（安装时）TensorFlow时生成的，位于Python库site-packages/tensorflow/python/ops/gen_math_ops.py中，随后又调用了tensorflow-1.1.0/tensorflow/core/kernels/下面的核函数实现。再例如，数组运算的代码位于tensorflow-1.1.0/tensorflow/python/ops/array_ops.py中，里面定义了concat、split、slice、size、rank等运算，每个函数都调用了gen_array_ops.py中的方法，这个文件也是在编译TensorFlow时生成的，位于Python库site-packages/tensorflow/python/ops/gen_array_ops.py中，随后又调用了tensorflow-1.1.0/tensorflow/core/kernels/下面的核函数实现。

### 4.3.3 其他概念

除了边和节点，TensorFlow还涉及其他一些概念，如图、会话、设备、变量、内核等。下面就分别介绍一下。

**1. 图**

把操作任务描述成有向无环图。那么，如何构建一个图呢？构建图的第一步是创建各个节点。具体如下：

In [4]:
import tensorflow as tf

# 创建一个常量运算操作，产生一个1x2矩阵
matrix1 = tf.constant([[3., 3.]])

# 创建另外一个常量运算操作，产生一个2x1矩阵
matrix2 = tf.constant([[2.], [2.]])

# 创建一个矩阵乘法运算，把matrix1和matrix2作为输入
# 返回值product代表矩阵乘法的结果
product = tf.matmul(matrix1, matrix2)

**2. 会话**

启动图的第一步是创建一个Session对象。会话（session）提供在图中执行操作的一些方法。一般的模式是，建立会话，此时会生成一张空图，在会话中添加节点和边，形成一张图，然后执行。

要创建一张图并运行操作的类，在Python的API中使用tf.Session，在C++的API中使用tensorflow::Session。示例如下：

In [5]:
with tf.Session() as sess:
    result = sess.run([product])
    print(result)

[array([[12.]], dtype=float32)]


在调用Session对象的run()方法来执行图时，传入一些Tensor，这个过程叫填充（feed）；返回的结果类型根据输入的类型而定，这个过程叫取回（fetch）。

与会话相关的源代码位于tensorflow-1.1.0/tensorflow/python/client/session.py。

会话是图交互的一个桥梁，一个会话可以有多个图，会话可以修改图的结构，也可以往图中注入数据进行计算。因此，会话主要有两个API接口：Extend和Run。Extend操作是在Graph中添加节点和边，Run操作是输入计算的节点和填充必要的数据后，进行计算，并输出运算结果。

**3. 设备**

设备（device）是指一块可以用来运算并且拥有自己的地址空间的硬件，如GPU和CPU。TensorFlow为了实现分布式执行操作，充分利用计算资源，可以明确指定操作在哪个设备上执行。具体如下：

In [ ]:
with tf.Session() as sess:
    # 指定在第二个gpu上运行
    with tf.device("/gpu:1"):
        matrix1 = tf.constant([[3., 3.]])
        matrix2 = tf.constant([[2.], [2.]])
        product = tf.matmul(matrix1, matrix2)

与设备相关的源代码位于tensorflow-1.1.0/tensorflow/python/framework/device.py。

**4. 变量**

变量（variable）是一种特殊的数据，它在图中有固定的位置，不像普通张量那样可以流动。例如，创建一个变量张量，使用tf.Variable()构造函数，这个构造函数需要一个初始值，初始值的形状和类型决定了这个变量的形状和类型：

In [6]:
# 创建一个变量，初始化为标量0
state = tf.Variable(0, name="counter")

创建一个常量张量：

In [7]:
input1 = tf.constant(3.0)

TensorFlow还提供了填充机制，可以在构建图时使用tf.placeholder()临时替代任意操作的张量，在调用Session对象的run()方法去执行图时，使用填充数据作为调用的参数，调用结束后，填充数据就消失。代码示例如下：

In [8]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)
with tf.Session() as sess:
    print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))
    # 输出 [array([14.], dtype=float32)]

[array([14.], dtype=float32)]


与变量相关的源代码位于tensorflow/tensorflow/python/ops/variables.py。

**5. 内核**

我们知道操作（operation）是对抽象操作（如matmul或者add）的一个统称，而内核（kernel）则是能够运行在特定设备（如CPU、GPU）上的一种对操作的实现。因此，同一个操作可能会对应多个内核。

当自定义一个操作时，需要把新操作和内核通过注册的方式添加到系统中。4.10节会用一个示例来讲解如何自定义一个操作。

## 4.4 常用API

了解TensorFlow的API有助于在应用时得心应手。下面介绍的是常用API，在后面的示例中基本上都会用到。这里主要介绍基于Python的API，基于其他语言的API也大同小异，最重要的是理解API的功能及其背后的原理。

### 4.4.1 图、操作和张量

TensorFlow的计算表现为数据流图，所以tf.Graph类中包含一系列表示计算的操作对象（tf.Operation），以及在操作之间流动的数据——张量对象（tf.Tensor）。与图相关的API均位于tf.Graph类中，参见表4-3。

![表4-3](4-attach/t4-3.png)

tf.Operation类代表图中的一个节点，用于计算张量数据。该类型由节点构造器（如tf.matmul()或者Graph.create_op()）产生。例如，```c = tf.matmul(a, b)```创建一个Operation类，其类型为MatMul的操作类。与操作相关的API均位于tf.Operation类中，参见表4-4。

![表4-4](4-attach/t4-4a.png)
![表4-4](4-attach/t4-4b.png)

tf.Tensor类是操作输出的符号句柄，它不包含操作输出的值，而是提供了一种在tf.Session中计算这些值的方法。这样就可以在操作之间构建一个数据流连接，使TensorFlow能够执行一个表示大量多步计算的图形。与张量相关的API均位于tf.Tensor类中，参见表4-5。

![表4-5](4-attach/t4-5.png)

### 4.4.2 可视化

在第3章中，我们讲解了可视化面板的功能，但如何编写可视化的程序呢？可视化时，需要在程序中给必要的节点添加摘要（summary），摘要会收集该节点的数据，并标记上第几步、时间戳等标识，写入事件文件（event file）中。tf.summary.FileWriter类用于在目录中创建事件文件，并且向文件中添加摘要和事件，用来在TensorBoard中展示。9.3节将详细讲解可视化的过程。

表4-6给出了可视化常用的API操作。

![表4-6](4-attach/t4-6a.png)
![表4-6](4-attach/t4-6b.png)